In [ ]:
#step-1)Input sanitization
import pandas as pd
import re

# Step 1: Load the dataset
df = pd.read_csv("/content/employee_records.csv")
print("Columns in dataset:", df.columns)
df['query'] = df.apply(lambda x: f"SHOW me emp in {x['Department']} dept under {x['Position']}", axis=1)
#Normalization function
def normalize_query(query):
    """
    Normalize queries by:
    - Lowercasing
    - Removing punctuation
    - Rewriting common abbreviations / synonyms
    - Removing extra spaces
    """
    query = query.lower()
    query = re.sub(r'[^a-z0-9\s]', '', query)
    # Replace common abbreviations / synonyms
    replacements = {
        'emp': 'employee',
        'dept': 'department',
        'show me': 'get',
        'list of': 'get',
        'give me': 'get'
    }
    for k, v in replacements.items():
        query = query.replace(k, v)
    query = re.sub(r'\s+', ' ', query).strip()    # remove extra spaces
    return query

# Apply normalization
df['normalized_query'] = df['query'].apply(normalize_query)

# Display Before vs After (first 15 queries)
print("BEFORE vs AFTER QUERY PREPROCESSING\n")
for i, row in enumerate(df.head(15).itertuples(), start=1):
    print(f"{i}. Before : {row.query}")
    print(f"   After  : {row.normalized_query}\n")


Columns in dataset: Index(['Employee_ID', 'Employee_Name', 'Age', 'Country', 'Department',
       'Position', 'Salary', 'Joining_Date'],
      dtype='object')
BEFORE vs AFTER QUERY PREPROCESSING

1. Before : SHOW me emp in HR dept under Analyst
   After  : get employee in hr department under analyst

2. Before : SHOW me emp in Marketing dept under Executive
   After  : get employee in marketing department under executive

3. Before : SHOW me emp in Finance dept under Developer
   After  : get employee in finance department under developer

4. Before : SHOW me emp in Support dept under Analyst
   After  : get employee in support department under analyst

5. Before : SHOW me emp in Support dept under Consultant
   After  : get employee in support department under consultant

6. Before : SHOW me emp in HR dept under Executive
   After  : get employee in hr department under executive

7. Before : SHOW me emp in Finance dept under Assistant
   After  : get employee in finance department und

In [ ]:
#Step-2)RBAC role expansion
import pandas as pd
df = pd.read_csv("/content/employee_records.csv")
import re
def normalize_query(query):
    query = query.lower()
    query = re.sub(r'[^a-z0-9\s]', '', query)
    replacements = {
        'emp': 'employee',
        'dept': 'department',
        'show me': 'get',
        'list of': 'get',
        'give me': 'get'
    }
    for k, v in replacements.items():
        query = query.replace(k, v)
    query = re.sub(r'\s+', ' ', query).strip()
    return query

df['query'] = df.apply(lambda x: f"Show me emp in {x['Department']} dept under {x['Position']}", axis=1)
df['normalized_query'] = df['query'].apply(normalize_query)

#  Define RBAC roles, permissions, and inheritance
roles = {
    "Admin": {"permissions": ["view", "edit", "delete"], "inherits": []},
    "HR": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
    "Manager": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
    "Employee": {"permissions": ["view"], "inherits": []}
}

# Function to get permissions including inheritance
def get_permissions(role_name):
    role = roles.get(role_name, {})
    perms = set(role.get("permissions", []))
    for parent in role.get("inherits", []):
        perms.update(get_permissions(parent))
    return perms

#  Assign RBAC roles based on Position
def map_role(position):
    position = position.lower()
    if "admin" in position:
        return "Admin"
    elif "hr" in position:
        return "HR"
    elif "manager" in position or "lead" in position:
        return "Manager"
    else:
        return "Employee"

df['role'] = df['Position'].apply(map_role)
df['permissions'] = df['role'].apply(get_permissions)

# Display example queries and RBAC permissions
print("QUERY | NORMALIZED | ROLE | PERMISSIONS\n")
for i, row in enumerate(df.head(15).itertuples(), start=1):
    print(f"{i}. Query      : {row.query}")
    print(f"   Normalized : {row.normalized_query}")
    print(f"   Role       : {row.role}")
    print(f"   Permissions: {row.permissions}\n")


QUERY | NORMALIZED | ROLE | PERMISSIONS

1. Query      : Show me emp in HR dept under Analyst
   Normalized : get employee in hr department under analyst
   Role       : Employee
   Permissions: {'view'}

2. Query      : Show me emp in Marketing dept under Executive
   Normalized : get employee in marketing department under executive
   Role       : Employee
   Permissions: {'view'}

3. Query      : Show me emp in Finance dept under Developer
   Normalized : get employee in finance department under developer
   Role       : Employee
   Permissions: {'view'}

4. Query      : Show me emp in Support dept under Analyst
   Normalized : get employee in support department under analyst
   Role       : Employee
   Permissions: {'view'}

5. Query      : Show me emp in Support dept under Consultant
   Normalized : get employee in support department under consultant
   Role       : Employee
   Permissions: {'view'}

6. Query      : Show me emp in HR dept under Executive
   Normalized : get employ

In [ ]:
#RBAC filtering chunks
import pandas as pd
import re
df = pd.read_csv("/content/employee_records.csv")
def normalize_query(query):
    query = query.lower()
    query = re.sub(r'[^a-z0-9\s]', '', query)
    query = query.replace('show me', 'get')
    query = query.replace('list of', 'get')
    query = query.replace('give me', 'get')

    # Then word replacements with word boundaries
    query = re.sub(r'\bemp\b', 'employee', query)
    query = re.sub(r'\bdept\b', 'department', query)

    query = re.sub(r'\s+', ' ', query).strip()
    return query
df['query'] = df.apply(lambda x: f"Show me emp in {x['Department']} dept under {x['Position']}", axis=1)
df['normalized_query'] = df['query'].apply(normalize_query)

#  RBAC roles and permissions
roles = {
    "Admin": {"permissions": ["view", "edit", "delete"], "inherits": []},
    "HR": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
    "Manager": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
    "Employee": {"permissions": ["view"], "inherits": []}
}

def get_permissions(role_name):
    role = roles.get(role_name, {})
    perms = set(role.get("permissions", []))
    for parent in role.get("inherits", []):
        perms.update(get_permissions(parent))
    return perms

def map_role(position):
    position = position.lower()
    if "admin" in position:
        return "Admin"
    elif "hr" in position:
        return "HR"
    elif "manager" in position or "lead" in position:
        return "Manager"
    else:
        return "Employee"

df['role'] = df['Position'].apply(map_role)
df['permissions'] = df['role'].apply(get_permissions)

# RBAC filtering function
def rbac_filter(user_role, normalized_query, dataset):
    """
    Returns only the rows (chunks) that the user's role is allowed to see.
    Example: Employee can see only their own department, Manager can see their team, Admin can see all.
    """
    # Admin can see all
    if user_role == "Admin":
        return dataset

    # Manager can see rows in their department
    if user_role == "Manager":
        # extract department from query if present
        dept_match = re.search(r"department (\w+)", normalized_query)
        if dept_match:
            dept = dept_match.group(1).upper()
            return dataset[dataset['Department'].str.upper() == dept]
        else:
            return dataset

    # HR can see all employees
    if user_role == "HR":
        return dataset

    # Employee can see only their own record
    if user_role == "Employee":
        # if employee name is mentioned in query
        # Modified regex to capture full name (e.g., 'Daniel Taylor' instead of just 'Daniel')
        name_match = re.search(r"employee\s+(.+)", normalized_query)
        if name_match:
            name = name_match.group(1).lower()
            # Assuming 'Employee_Name' is the column containing employee names
            return dataset[dataset['Employee_Name'].str.lower() == name]
        else:
            # fallback: no rows
            return pd.DataFrame(columns=dataset.columns)

# Example usage for user_index = 0 (Daniel Taylor, Employee Role)
user_index = 0
user_role = df.loc[user_index, 'role']
employee_name_for_test = df.loc[user_index, 'Employee_Name']
query_for_demo = f"Show me information for employee {employee_name_for_test}"
normalized_query_for_demo = normalize_query(query_for_demo)
query = query_for_demo
normalized_query = normalized_query_for_demo
filtered_chunks = rbac_filter(user_role, normalized_query, df)
print(f"USER ROLE: {user_role}")
print(f"ORIGINAL QUERY: {query}")
print(f"NORMALIZED QUERY: {normalized_query}")
print(f"FILTERED CHUNKS (rows visible to user):\n", filtered_chunks.head())
print("\n--- Comparison with Original Example Query ---")
original_query_text = df.loc[user_index, 'query']
original_normalized_query_text = df.loc[user_index, 'normalized_query']
original_filtered_chunks = rbac_filter(user_role, original_normalized_query_text, df)
print(f"ORIGINAL QUERY (from df column): {original_query_text}")
print(f"NORMALIZED QUERY (from df column): {original_normalized_query_text}")
print(f"FILTERED CHUNKS (rows visible to user with original query):\n", original_filtered_chunks.head())

USER ROLE: Employee
ORIGINAL QUERY: Show me information for employee Daniel Taylor
NORMALIZED QUERY: get information for employee daniel taylor
FILTERED CHUNKS (rows visible to user):
      Employee_ID  Employee_Name  Age Country   Department   Position  \
0              1  Daniel Taylor   25      UK           HR    Analyst   
63            64  Daniel Taylor   40   Japan      Support  Executive   
190          191  Daniel Taylor   47   India    Marketing    Analyst   
197          198  Daniel Taylor   28  Canada      Support  Executive   
351          352  Daniel Taylor   52  Canada  Engineering  Executive   

        Salary Joining_Date                                            query  \
0    142278.32   2023-06-04             Show me emp in HR dept under Analyst   
63    40581.90   2015-08-23      Show me emp in Support dept under Executive   
190  130916.79   2024-04-25      Show me emp in Marketing dept under Analyst   
197   41049.89   2019-11-26      Show me emp in Support dept u

In [ ]:
#step-4)KNN brute force candidate
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
def rbac_filter(user_role, normalized_query, dataset):
    """
    Relaxed RBAC filtering for testing KNN.
    Admin/HR see all.
    Manager/Employee see all for demo purposes
    """
    if user_role in ["Admin", "HR", "Manager", "Employee"]:
        return dataset.copy()
    else:
        return pd.DataFrame(columns=dataset.columns)

def knn_candidates(user_role, normalized_query, dataset, K=5):
    """
    Returns top K candidate chunks based on query similarity after RBAC filtering.
    """
    filtered_chunks = rbac_filter(user_role, normalized_query, dataset)

    if filtered_chunks.empty:
        print("No chunks available for this user based on RBAC.")
        return pd.DataFrame()

    # Prepare text for TF-IDF (Position + Department)
    filtered_chunks['text'] = filtered_chunks['Position'].astype(str) + " " + filtered_chunks['Department'].astype(str)
    corpus = filtered_chunks['text'].tolist()

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer()
    chunk_vectors = vectorizer.fit_transform(corpus)

    # Vectorize normalized query
    query_vector = vectorizer.transform([normalized_query])

    # Cosine similarity (brute force)
    similarities = cosine_similarity(query_vector, chunk_vectors).flatten()

    # Get top K candidates
    top_k_indices = similarities.argsort()[::-1][:K]

    top_candidates = filtered_chunks.iloc[top_k_indices].copy()
    top_candidates['similarity'] = similarities[top_k_indices]

    return top_candidates


# Load dataset
df = pd.read_csv("/content/employee_records.csv")

# Example user role and query
user_role = "Admin"
normalized_query = "get employee department CSE"

# Get top 5 candidates
top_candidates = knn_candidates(user_role, normalized_query, df, K=5)

# Display results
print("\nTop Candidate Chunks:\n")
print(top_candidates[['Employee_Name', 'Department', 'Position', 'similarity']])


Top Candidate Chunks:

      Employee_Name   Department    Position  similarity
16     Lily Jackson    Marketing   Assistant         0.0
17  Daniel Anderson  Engineering  Consultant         0.0
18      Ethan Davis           HR  Consultant         0.0
19   Lily Hernandez      Support     Analyst         0.0
20     Sophia Brown      Finance  Consultant         0.0


In [ ]:
#step-5)merging query results along with variance
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re


def rbac_filter(user_role, normalized_query, dataset):
    """
    Relaxed RBAC filter: all roles see all rows for testing.
    """
    return dataset.copy()


def knn_candidates(user_role, normalized_query, dataset, K=5):
    """
    Returns top K candidate chunks using TF-IDF similarity.
    Extracts only relevant keywords from query that exist in dataset columns.
    """
    # RBAC filtering
    filtered_chunks = rbac_filter(user_role, normalized_query, dataset)
    if filtered_chunks.empty:
        print("No chunks available for this user based on RBAC.")
        return pd.DataFrame()

    filtered_chunks = filtered_chunks.copy()

    # Combine text columns for TF-IDF
    filtered_chunks['text'] = filtered_chunks['Position'].astype(str) + " " + filtered_chunks['Department'].astype(str)
    corpus = filtered_chunks['text'].tolist()

    # Extract keywords from query that exist in dataset
    keywords = []
    for col in ['Position', 'Department']:
        for value in dataset[col].unique():
            if str(value).lower() in normalized_query.lower():
                keywords.append(str(value))
    # Use only keywords from query for vectorization
    query_for_tfidf = " ".join(keywords) if keywords else normalized_query

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    chunk_vectors = vectorizer.fit_transform(corpus)
    query_vector = vectorizer.transform([query_for_tfidf])

    # Cosine similarity
    similarities = cosine_similarity(query_vector, chunk_vectors).flatten()

    # Top K candidates
    top_k_indices = similarities.argsort()[::-1][:K]
    top_candidates = filtered_chunks.iloc[top_k_indices].copy()
    top_candidates['similarity'] = similarities[top_k_indices]

    return top_candidates
df = pd.read_csv("/content/employee_records.csv")

# Example user role and normalized query
user_role = "Employee"
normalized_query = "get employee in hr department under analyst"

top_candidates = knn_candidates(user_role, normalized_query, df, K=5)

# Display results
print("\nTop Candidate Chunks:\n")
print(top_candidates[['Employee_Name', 'Department', 'Position', 'similarity']])


Top Candidate Chunks:

         Employee_Name Department Position  similarity
4987   Mason Hernandez         HR  Analyst         1.0
23997       Lily Brown         HR  Analyst         1.0
23994       Lily Brown         HR  Analyst         1.0
23993      Emma Thomas         HR  Analyst         1.0
2716    Daniel Jackson         HR  Analyst         1.0


In [ ]:
#step-6)deduplication
import pandas as pd
results1 = top_candidates.copy()
results2 = top_candidates.sample(frac=1, random_state=42)
results3 = top_candidates.sample(frac=1, random_state=24)
merged = pd.concat([results1, results2, results3], ignore_index=True)
# If a candidate appears multiple times, calculate variance of similarity scores
variance_df = merged.groupby(['Employee_Name', 'Department', 'Position'])['similarity'].agg(['mean', 'var']).reset_index()

# Fill NaN variance (if only one occurrence) with 0
variance_df['var'] = variance_df['var'].fillna(0)
# Ranking

# Rank by mean similarity descending, then by lowest variance (more consistent scores first)
variance_df['rank'] = variance_df.sort_values(['mean', 'var'], ascending=[False, True]).reset_index().index + 1

# =========================
# Display Final Ranked Results
# =========================
final_ranked = variance_df.sort_values('rank')
print("\nFinal Merged & Ranked Results:\n")
print(final_ranked[['rank', 'Employee_Name', 'Department', 'Position', 'mean', 'var']])


Final Merged & Ranked Results:

   rank    Employee_Name Department Position  mean  var
0     1   Daniel Jackson         HR  Analyst   1.0  0.0
1     2      Emma Thomas         HR  Analyst   1.0  0.0
2     3       Lily Brown         HR  Analyst   1.0  0.0
3     4  Mason Hernandez         HR  Analyst   1.0  0.0


In [ ]:
#step-7)Thresholding
import pandas as pd
deduplicated = final_ranked.drop_duplicates(subset=['Employee_Name', 'Department', 'Position'], keep='first')


deduplicated['final_rank'] = range(1, len(deduplicated) + 1)


print("\nFinal Deduplicated & Ranked Results:\n")
print(deduplicated[['final_rank', 'Employee_Name', 'Department', 'Position', 'mean', 'var']])



Final Deduplicated & Ranked Results:

   final_rank    Employee_Name Department Position  mean  var
0           1   Daniel Jackson         HR  Analyst   1.0  0.0
1           2      Emma Thomas         HR  Analyst   1.0  0.0
2           3       Lily Brown         HR  Analyst   1.0  0.0
3           4  Mason Hernandez         HR  Analyst   1.0  0.0


In [ ]:
#step-8)combined pipeline function
import pandas as pd


SIMILARITY_THRESHOLD = 0.2


thresholded_results = deduplicated[deduplicated['mean'] >= SIMILARITY_THRESHOLD].copy()


thresholded_results['final_rank'] = range(1, len(thresholded_results) + 1)


print("\nFinal Results after Thresholding:\n")
print(thresholded_results[['final_rank', 'Employee_Name', 'Department', 'Position', 'mean', 'var']])


Final Results after Thresholding:

   final_rank    Employee_Name Department Position  mean  var
0           1   Daniel Jackson         HR  Analyst   1.0  0.0
1           2      Emma Thomas         HR  Analyst   1.0  0.0
2           3       Lily Brown         HR  Analyst   1.0  0.0
3           4  Mason Hernandez         HR  Analyst   1.0  0.0


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def rbac_knn_pipeline(user_role, raw_query, dataset, K=5, similarity_threshold=0.2):
    """
    Full RBAC + KNN pipeline (Steps 1-7)
    Returns final deduplicated and thresholded ranked candidates.
    """

    # -----------------------
    # Step 1: Normalize query
    # -----------------------
    def normalize_query(query):
        query = query.lower()
        query = re.sub(r'[^a-z0-9\s]', '', query)
        replacements = {
            'emp': 'employee',
            'dept': 'department',
            'show me': 'get',
            'list of': 'get',
            'give me': 'get'
        }
        for k, v in replacements.items():
            query = query.replace(k, v)
        query = re.sub(r'\s+', ' ', query).strip()
        return query

    normalized_query = normalize_query(raw_query)

    # -----------------------
    # Step 2: RBAC Role Expansion
    # -----------------------
    roles = {
        "Admin": {"permissions": ["view", "edit", "delete"], "inherits": []},
        "HR": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
        "Manager": {"permissions": ["view", "edit"], "inherits": ["Employee"]},
        "Employee": {"permissions": ["view"], "inherits": []}
    }

    def get_permissions(role_name):
        role = roles.get(role_name, {})
        perms = set(role.get("permissions", []))
        for parent in role.get("inherits", []):
            perms.update(get_permissions(parent))
        return perms

    # For demo, assign permissions to user_role
    user_permissions = get_permissions(user_role)

    # -----------------------
    # Step 3: RBAC Filtering Chunks
    # -----------------------
    def rbac_filter(user_role, normalized_query, dataset):
        """
        Relaxed RBAC for testing: all roles see full dataset.
        You can tighten this based on your RBAC rules.
        """
        return dataset.copy()

    filtered_chunks = rbac_filter(user_role, normalized_query, dataset)

    if filtered_chunks.empty:
        print("No chunks available for this user based on RBAC.")
        return pd.DataFrame()

    # -----------------------
    # Step 4: KNN Candidate Retrieval
    # -----------------------
    filtered_chunks = filtered_chunks.copy()
    filtered_chunks['text'] = filtered_chunks['Position'].astype(str) + " " + filtered_chunks['Department'].astype(str)
    corpus = filtered_chunks['text'].tolist()

    # Extract keywords from query present in dataset columns
    keywords = []
    for col in ['Position', 'Department']:
        for value in dataset[col].unique():
            if str(value).lower() in normalized_query.lower():
                keywords.append(str(value))
    query_for_tfidf = " ".join(keywords) if keywords else normalized_query

    # TF-IDF + cosine similarity
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    chunk_vectors = vectorizer.fit_transform(corpus)
    query_vector = vectorizer.transform([query_for_tfidf])
    similarities = cosine_similarity(query_vector, chunk_vectors).flatten()

    filtered_chunks['similarity'] = similarities
    top_candidates = filtered_chunks.nlargest(K, 'similarity')

    # -----------------------
    # Step 5: Merge, Variance, Ranking
    # -----------------------
    merged = pd.concat([top_candidates, top_candidates.sample(frac=1), top_candidates.sample(frac=1)], ignore_index=True)
    variance_df = merged.groupby(['Employee_Name', 'Department', 'Position'])['similarity'].agg(['mean', 'var']).reset_index()
    variance_df['var'] = variance_df['var'].fillna(0)
    variance_df['rank'] = variance_df.sort_values(['mean', 'var'], ascending=[False, True]).reset_index().index + 1

    # -----------------------
    # Step 6: Deduplication
    # -----------------------
    deduplicated = variance_df.drop_duplicates(subset=['Employee_Name', 'Department', 'Position'], keep='first').reset_index(drop=True)
    deduplicated['final_rank'] = range(1, len(deduplicated) + 1)

    # -----------------------
    # Step 7: Thresholding
    # -----------------------
    thresholded_results = deduplicated[deduplicated['mean'] >= similarity_threshold].copy()
    thresholded_results['final_rank'] = range(1, len(thresholded_results) + 1)

    return thresholded_results[['final_rank', 'Employee_Name', 'Department', 'Position', 'mean', 'var']]


df = pd.read_csv("/content/employee_records.csv")  # replace with your path

# Test the pipeline
user_role = "Admin"
raw_query = "Show me emp in HR dept under Analyst" # Updated query to use existing terms
final_results = rbac_knn_pipeline(user_role, raw_query, df, K=5, similarity_threshold=0.1)

print("\nFinal RBAC + KNN Pipeline Results:\n")
print(final_results)


Final RBAC + KNN Pipeline Results:

   final_rank  Employee_Name Department Position  mean  var
0           1   Daniel Davis         HR  Analyst   1.0  0.0
1           2  Daniel Taylor         HR  Analyst   1.0  0.0
2           3    Emma Wilson         HR  Analyst   1.0  0.0
3           4    Lucas Davis         HR  Analyst   1.0  0.0
4           5     Mia Taylor         HR  Analyst   1.0  0.0


In [ ]:
#secure and debugging
import pandas as pd

df = pd.read_csv("/content/employee_records.csv")
print(df.columns)
print(df.head(2))


Index(['Employee_ID', 'Employee_Name', 'Age', 'Country', 'Department',
       'Position', 'Salary', 'Joining_Date'],
      dtype='object')
   Employee_ID  Employee_Name  Age Country Department   Position     Salary  \
0            1  Daniel Taylor   25      UK         HR    Analyst  142278.32   
1            2    Ethan Brown   44   India  Marketing  Executive   98549.20   

  Joining_Date  
0   2023-06-04  
1   2018-01-13  


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ===============================
# SAFE COLUMN DETECTION
# ===============================
def detect_column(possible_names, df):
    for col in df.columns:
        if col.lower() in possible_names:
            return col
    raise ValueError(f"Required column not found: {possible_names}")

# ===============================
# MAIN PIPELINE
# ===============================
def guaranteed_pipeline(query, df, K=5):

    # --- Normalize query ---
    query = query.lower()
    query = re.sub(r'[^a-z0-9\s]', '', query)

    # --- Detect correct columns ---
    name_col = detect_column({"name", "employee_name"}, df)
    dept_col = detect_column({"department", "dept"}, df)
    pos_col  = detect_column({"position", "job_title", "role"}, df)

    print("✔ Columns detected:", name_col, dept_col, pos_col)

    # --- Build text corpus ---
    df = df.copy()
    df["text"] = df[pos_col].astype(str) + " " + df[dept_col].astype(str)

    # --- TF-IDF ---
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(df["text"])
    query_vec = vectorizer.transform([query])

    similarities = cosine_similarity(query_vec, vectors).flatten()
    df["similarity"] = similarities

    # --- Always return top K ---
    result = df.sort_values("similarity", ascending=False).head(K)

    print("\n✅ OUTPUT (GUARANTEED):")
    print(result[[name_col, dept_col, pos_col, "similarity"]])

    return result


In [ ]:
df = pd.read_csv("/content/employee_records.csv")

guaranteed_pipeline(
    query="software engineer cse",
    df=df
)

✔ Columns detected: Employee_Name Department Position

✅ OUTPUT (GUARANTEED):
          Employee_Name   Department    Position  similarity
29983        Emma Davis      Finance   Assistant         0.0
29982  Daniel Hernandez           HR   Executive         0.0
29981      Daniel Davis  Engineering  Consultant         0.0
29980      Sophia Davis    Marketing     Manager         0.0
29979         Ava Moore      Finance     Analyst         0.0


,Employee_ID,Employee_Name,Age,Country,Department,Position,Salary,Joining_Date,text,similarity
29983,29984,Emma Davis,57,India,Finance,Assistant,51470.76,2022-08-13,Assistant Finance,0.0
29982,29983,Daniel Hernandez,39,France,HR,Executive,70460.69,2025-01-13,Executive HR,0.0
29981,29982,Daniel Davis,31,Brazil,Engineering,Consultant,110659.04,2017-08-29,Consultant Engineering,0.0
29980,29981,Sophia Davis,29,Australia,Marketing,Manager,120866.27,2020-08-28,Manager Marketing,0.0
29979,29980,Ava Moore,52,India,Finance,Analyst,145634.18,2015-10-17,Analyst Finance,0.0


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 1️⃣ Load dataset
df = pd.read_csv("/content/employee_records.csv")

# 2️⃣ Create corpus from dataset (VERY IMPORTANT)
corpus = (
    df["Position"].astype(str) + " " + df["Department"].astype(str)
).tolist()

# 3️⃣ Define query
query = "show me software engineer from cse department"

# 4️⃣ Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 5️⃣ Encode corpus and query
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
query_embedding = model.encode(query, convert_to_tensor=True)

# 6️⃣ Compute cosine similarity
similarity_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

# 7️⃣ Get top 5 matches
top_k = 5
top_results = similarity_scores.topk(k=top_k)

# 8️⃣ Display results
print("\nTop Semantic Matches:\n")
for score, idx in zip(top_results.values, top_results.indices):
    print(
        f"Name: {df.iloc[idx.item()]['Employee_Name']} | " # Corrected idx to idx.item() and 'Name' to 'Employee_Name'
        f"Department: {df.iloc[idx.item()]['Department']} | " # Corrected idx to idx.item()
        f"Position: {df.iloc[idx.item()]['Position']} | " # Corrected idx to idx.item()
        f"Score: {score:.3f}"
    )



Top Semantic Matches:

Name: Logan Martinez | Department: Engineering | Position: Developer | Score: 0.530
Name: Daniel Jackson | Department: Engineering | Position: Developer | Score: 0.530
Name: Emma Thomas | Department: Engineering | Position: Developer | Score: 0.530
Name: Ethan Hernandez | Department: Engineering | Position: Developer | Score: 0.530
Name: Mason Anderson | Department: Engineering | Position: Developer | Score: 0.530
